# TSLL MA200 Strategy - Daily Signal Checker

This notebook checks today's market conditions to determine if it's a **BUY** or **SELL** signal.

## Strategy Rules:
- **BUY TSLL** when: TSLA > MA200 × 1.04 AND TSLA daily loss >= 1%
- **SELL** when: TSLA < MA200 × 0.97

## Step 1: Setup

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Strategy parameters
MA_PERIOD = 200
BUY_THRESHOLD = 1.04
SELL_THRESHOLD = 0.97
DAILY_LOSS_THRESHOLD = -0.01

print(f"Strategy Rules:")
print(f"  BUY when: TSLA > MA{MA_PERIOD} × {BUY_THRESHOLD} AND daily loss >= 1%")
print(f"  SELL when: TSLA < MA{MA_PERIOD} × {SELL_THRESHOLD}")
print(f"\nChecking signals as of: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## Step 2: Fetch Latest Data

In [ ]:
# Fetch enough data for MA200 calculation
start_date = datetime.now() - timedelta(days=365)

print("Fetching TSLA data...")
tsla = yf.download('TSLA', start=start_date, progress=False)

print("Fetching TSLL data...")
tsll = yf.download('TSLL', start=start_date, progress=False)

# Handle multi-level columns
if isinstance(tsla.columns, pd.MultiIndex):
    tsla.columns = tsla.columns.get_level_values(0)
if isinstance(tsll.columns, pd.MultiIndex):
    tsll.columns = tsll.columns.get_level_values(0)

print(f"\nData fetched: {len(tsla)} trading days")
print(f"Latest date: {tsla.index[-1].strftime('%Y-%m-%d')}")

## Step 3: Calculate Indicators

In [ ]:
# Calculate MA200
tsla['MA200'] = tsla['Close'].rolling(window=MA_PERIOD).mean()

# Calculate daily return
tsla['Daily_Return'] = tsla['Close'].pct_change()

# Calculate thresholds
tsla['Buy_Level'] = tsla['MA200'] * BUY_THRESHOLD
tsla['Sell_Level'] = tsla['MA200'] * SELL_THRESHOLD

# Get today's values
today = tsla.iloc[-1]
yesterday = tsla.iloc[-2]

print("Indicators calculated!")

## Step 4: Today's Market Data

In [ ]:
# Display today's key metrics
print("="*60)
print(f"TODAY'S MARKET DATA ({tsla.index[-1].strftime('%Y-%m-%d')})")
print("="*60)
print(f"")
print(f"TSLA Close:        ${today['Close']:.2f}")
print(f"TSLA Daily Change: {today['Daily_Return']*100:+.2f}%")
print(f"")
print(f"MA200:             ${today['MA200']:.2f}")
print(f"Buy Level:         ${today['Buy_Level']:.2f}  (MA200 × {BUY_THRESHOLD})")
print(f"Sell Level:        ${today['Sell_Level']:.2f}  (MA200 × {SELL_THRESHOLD})")
print(f"")
print(f"TSLL Close:        ${tsll['Close'].iloc[-1]:.2f}")

## Step 5: Check Signal Conditions

In [ ]:
# Check conditions
tsla_above_buy_level = today['Close'] > today['Buy_Level']
tsla_daily_loss_met = today['Daily_Return'] <= DAILY_LOSS_THRESHOLD
tsla_below_sell_level = today['Close'] < today['Sell_Level']

# Determine signal
buy_signal = tsla_above_buy_level and tsla_daily_loss_met
sell_signal = tsla_below_sell_level

print("="*60)
print("CONDITION CHECK")
print("="*60)
print(f"")
print(f"BUY CONDITIONS:")
print(f"  [{'✓' if tsla_above_buy_level else '✗'}] TSLA (${today['Close']:.2f}) > Buy Level (${today['Buy_Level']:.2f})")
print(f"  [{'✓' if tsla_daily_loss_met else '✗'}] Daily loss ({today['Daily_Return']*100:+.2f}%) >= 1%")
print(f"")
print(f"SELL CONDITION:")
print(f"  [{'✓' if tsla_below_sell_level else '✗'}] TSLA (${today['Close']:.2f}) < Sell Level (${today['Sell_Level']:.2f})")

## Step 6: TODAY'S SIGNAL

In [ ]:
print("\n" + "="*60)
print("📊 TODAY'S SIGNAL")
print("="*60)

if buy_signal:
    print(f"\n  🟢 BUY TSLL @ ${tsll['Close'].iloc[-1]:.2f}")
    print(f"\n  Reason: TSLA is above MA200×1.04 AND dropped >= 1% today")
elif sell_signal:
    print(f"\n  🔴 SELL TSLL @ ${tsll['Close'].iloc[-1]:.2f}")
    print(f"\n  Reason: TSLA dropped below MA200×0.97")
else:
    print(f"\n  ⚪ HOLD / NO ACTION")
    print(f"\n  Neither buy nor sell conditions are met.")
    
    # Show what's missing for a buy signal
    if not tsla_above_buy_level:
        diff = today['Buy_Level'] - today['Close']
        pct = (diff / today['Close']) * 100
        print(f"\n  For BUY: TSLA needs to rise ${diff:.2f} (+{pct:.2f}%) to reach buy level")
    elif not tsla_daily_loss_met:
        print(f"\n  For BUY: Waiting for a -1% daily drop (today was {today['Daily_Return']*100:+.2f}%)")
    
    # Show distance to sell level
    if not tsla_below_sell_level:
        diff = today['Close'] - today['Sell_Level']
        pct = (diff / today['Close']) * 100
        print(f"  For SELL: TSLA needs to drop ${diff:.2f} (-{pct:.2f}%) to reach sell level")

print("\n" + "="*60)

## Step 7: Current Position Tracker

In [ ]:
# Determine current position based on recent history
# Look back to find the most recent buy or sell signal

def get_current_position(tsla_data):
    """Determine current position based on historical signals."""
    position = 0  # Start with no position
    last_action = None
    last_action_date = None
    last_action_price = None
    
    for i in range(len(tsla_data)):
        row = tsla_data.iloc[i]
        date = tsla_data.index[i]
        
        buy_cond = (row['Close'] > row['Buy_Level']) and (row['Daily_Return'] <= DAILY_LOSS_THRESHOLD)
        sell_cond = row['Close'] < row['Sell_Level']
        
        if position == 0 and buy_cond:
            position = 1
            last_action = 'BUY'
            last_action_date = date
            last_action_price = row['Close']
        elif position == 1 and sell_cond:
            position = 0
            last_action = 'SELL'
            last_action_date = date
            last_action_price = row['Close']
    
    return position, last_action, last_action_date, last_action_price

position, last_action, last_date, last_price = get_current_position(tsla)

print("="*60)
print("CURRENT POSITION STATUS")
print("="*60)
print(f"")
if position == 1:
    print(f"  Position: 🟢 LONG TSLL")
else:
    print(f"  Position: ⚪ CASH (No Position)")

if last_action:
    print(f"  Last Action: {last_action} on {last_date.strftime('%Y-%m-%d')} @ TSLA ${last_price:.2f}")
else:
    print(f"  Last Action: None in recent history")
print(f"")

## Step 8: Recent Price History

In [ ]:
# Show last 10 days
recent = tsla[['Close', 'Daily_Return', 'MA200', 'Buy_Level', 'Sell_Level']].tail(10).copy()
recent['Daily_Return'] = recent['Daily_Return'].apply(lambda x: f"{x*100:+.2f}%")
recent.columns = ['TSLA Close', 'Daily Chg', 'MA200', 'Buy Level', 'Sell Level']

print("Last 10 Trading Days:")
print(recent.to_string())

## Step 9: Quick Chart

In [ ]:
import matplotlib.pyplot as plt

# Plot last 60 days
plot_data = tsla.tail(60)

plt.figure(figsize=(14, 6))
plt.plot(plot_data.index, plot_data['Close'], label='TSLA', linewidth=2, color='black')
plt.plot(plot_data.index, plot_data['MA200'], label='MA200', linewidth=1.5, color='blue')
plt.plot(plot_data.index, plot_data['Buy_Level'], label='Buy Level', linewidth=1, linestyle='--', color='green')
plt.plot(plot_data.index, plot_data['Sell_Level'], label='Sell Level', linewidth=1, linestyle='--', color='red')

# Mark today
plt.scatter([plot_data.index[-1]], [plot_data['Close'].iloc[-1]], color='purple', s=150, zorder=5, label='Today')

plt.title(f'TSLA - Last 60 Days (Today: {tsla.index[-1].strftime("%Y-%m-%d")})', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

Run this notebook daily (after market close) to check for signals. 

**Remember:**
- BUY signal requires BOTH conditions: TSLA > MA200×1.04 AND daily drop >= 1%
- SELL signal triggers when TSLA < MA200×0.97
- Execute trades at next day's open to avoid look-ahead bias